<a href="https://colab.research.google.com/github/lenguyennhattung/MCPython-FreeCodeCamp/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load data
train_df = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
test_df = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])

# Convert labels to numerical (0 for ham, 1 for spam)
train_df['label'] = train_df['label'].map({'ham': 0, 'spam': 1})
test_df['label'] = test_df['label'].map({'ham': 0, 'spam': 1})

# Separate messages and labels
training_sentences = train_df['message'].tolist()
training_labels = train_df['label'].to_numpy()

testing_sentences = test_df['message'].tolist()
testing_labels = test_df['label'].to_numpy()

# Text preprocessing parameters
vocab_size = 10000 # Max number of words to keep, based on frequency
embedding_dim = 16
max_len = 120 # Max length of a sentence after padding (experiment with this)
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>" # Out of vocabulary token

# Initialize and fit the tokenizer on training data
tokenizer = keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

# Create sequences and pad them
training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = keras.preprocessing.sequence.pad_sequences(training_sequences, maxlen=max_len,
                                                             padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = keras.preprocessing.sequence.pad_sequences(testing_sequences, maxlen=max_len,
                                                            padding=padding_type, truncating=trunc_type)

# Convert to numpy arrays
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    tf.keras.layers.GlobalAveragePooling1D(), # or GlobalMaxPooling1D()
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # Binary classification output
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
num_epochs = 10 # Experiment with number of epochs
history = model.fit(training_padded, training_labels, epochs=num_epochs,
                    validation_data=(testing_padded, testing_labels), verbose=2)

In [ ]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
    tf.keras.layers.GlobalAveragePooling1D(), # or GlobalMaxPooling1D()
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # Binary classification output
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
num_epochs = 10 # Experiment with number of epochs
history = model.fit(training_padded, training_labels, epochs=num_epochs,
                    validation_data=(testing_padded, testing_labels), verbose=2)

In [ ]:
# Function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # Preprocess the input message in the same way as training data
  sequence = tokenizer.texts_to_sequences([pred_text])
  padded_sequence = keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_len,
                                                                 padding=padding_type, truncating=trunc_type)

  # Make prediction
  prediction_prob = model.predict(padded_sequence)[0][0] # Get the single probability value

  # Determine label based on a threshold (0.5 is standard for binary classification)
  if prediction_prob > 0.5:
    label = 'spam'
  else:
    label = 'ham'

  return [prediction_prob, label]

# Example usage of the prediction function
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
